# Week 11 problems

## Question 1: Dalitz Decays and Resonant Structure

### Learning objectives
In this question you will:

- Gain experience in using the Particle Data Group compilations of decay rates to determine relative branching fractions for different decay modes and to develop insight on decay kinematics. Using the example of $D_s^{+}\rightarrow K^+K^-\pi^{+}$
- review how Dalitz Plots can  be  used to understand resonant structure in hadron decays.


B-factories are a good place to study the decays of bottom hadrons.  These $e^+e^-$ colliders typically operate at a center-of-mass energy of 10.579 GeV, the mass of the $\Upsilon (4s)$.  This $b\overline b $ resonance is just above open bottom threshold and therefore decays (more than 96% of the time) to
a $B\overline B$ (there is not enough additional energy to produce even a single additional pion).

Consider the case $\Upsilon (4s)\rightarrow B^0\overline B^0$.  If the $\overline B^0$ is fully reconstructed, then we know that all the remaining particles must come from the decay of the $B^0$ and if the $B^0$ is fully reconstructed, then we know all the remaining particles must come from the deacy of the $\overline B^0$. This strategy can therefore be used to measure absolute branching fractions.

### 1a. 

Consider the production $e^+e^- \rightarrow  B\overline B$ followed by the
decay chain for the $B^0$ and $\overline{B^0}$.  Some fraction of the time, the $B^0$ will decay through the following decay chain:
\begin{eqnarray*}
B^0 & \rightarrow & D_s^{*+}\pi^-\\
D_s^{*+} & \rightarrow & D_s^+ \gamma\\
D_s^{+}  & \rightarrow & K^+K^- \pi^+
\end{eqnarray*}

Here the $D_s^{+}$ decay to $K^+K^-\pi^+$ can occur through several mechanisms, including:

\begin{eqnarray*}
D_s^{+} \rightarrow \phi \pi^{+} & \;\;{\rm followed\; by} & \; \phi\rightarrow K^{+} K^{-}\\
D_s^{+} \rightarrow  K^{*0} K^{+} & \;\;{\rm followed\; by} & K^{*0}\rightarrow K^{-} \pi^{+} {\rm or\ } K^{*0}\rightarrow K^{+} \pi^{-}\\ 
D_s^{+}  \rightarrow  K^+K^- \pi^+ & & {\rm three-body\; (non-resonant)}
\end{eqnarray*}

For each of the particles listed above, use the Particle Data Group (PDG) tables to find their mass and spin. (Note, each of the particles listed above will have an anti-particle with opposite charge and the same mass and spin as the particle.  The $\overline{B^0}$ can decay through a decay chain where all the particles above are replaced by their anti-particles). Please note that as there are two $B$-mesons in each event, you will need to consider decays of both the $B$ mesons to find all the final state particles.

Write your answer here

### 1b. 

The $D_s^{*+}$ is composed of a charm quark and an anti-strange quark.  There is a similar state consisting of a charm quark and an anti-down quark called the $D^{*+}$.
Use the results tabulated by the PDG to compare the branching fractions for  $D_s^{*+}\rightarrow D_s^+ \gamma$ and $D^{*+}\rightarrow D^+ \gamma$.  From the properties listed in the PDG, can you explain why are these so different?

Write your answer here

### 1c. 

As noted above,  $D_s\rightarrow K^+K^-\pi$ decay can occur through several possible intermediate resonance states.  In principle the decays to these should be combined by adding their decay amplitudes and then squaring the sum, since they can quantum mechanically interfere.  However, the resonant structure is still evident in the decay kinematics and it is possible to observe this resonant structure using a Dalitz plot. The file DsstarPiMC.dat contains a  simulated sample that corrresponds to the decay chain described above.

Some of our events contain a decay chain starting with $B^{0} \rightarrow  D_s^{*+}\pi^{-}$
where the $D_S^{*+}$ decays to a $D_s^+ \rightarrow K^+ K^- \pi^+$.  Other events contain a decay chain starting with $\overline{B^0} \rightarrow  D_s^{*-}\pi^+$
where the $D_S^{*-}$ decays to a $D_s^{+} \rightarrow K^{+} K^{-} \pi^{-}$.  In both these cases, the
final state particles are $K^{+}K^{-}\pi^{+}\pi^{-}$.  By selecting the $K^{+}K^{-}\pi^{\pm}$ combination with
invariant mass consistent with the $D_s^{\pm}$,  determine for each event which $\pi$ comes from the $D_s^\pm$ decay.  Using the $\pi^\pm$ you have selected, plot the invariant mass of the $K^\mp \pi^\pm$ combination in the region around the $K^{*0}$.  Do you see a peak in roughly the right place?  

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt

# Parse the input file.
file = "DsstarPiMC.dat"

# Information needed to interpret the input file structure
# The order of variables is explained in the metadata at the top of the file
pMc = 0              # Magnitude of (three)-momentum of the particle in the lab frame
massMc=1             # mass of the particle
thetaMc=2            # polar angle of the particle
phiMc=3              # azimuthal angle of the  particle
idMc=4               # PDG ID of the particle (See http://www-pdg.lbl.gov/2018/reviews/rpp2018-rev-monte-carlo-numbering.pdf) 
mothMc=5             # index to the mother of the particle (the row corresponding to the mother, where
                     # the numbering of rows for each event starts at zero
nDauMc=6             # number of daughters the particle has
#
# The pdg IDs of the particles we care about
pdgIDKplus=0 # update this value
pdgIDKminus=0 # update this value
pdgIDPiplus=0 # update this value
pdgIDPiminus=0 # update this value
DsMass = 0 # in GeV # update this value

# This loop will process the file
inEvent = False
events = []
for line in open(file,"r"):
    line = line.strip()
    info = line.split(",")    
    if info[0] =="<event>":
        inEvent = True
        particles = []
    elif inEvent and (info[0] == "</event>"):
        events.append(particles)
        inEvent = False
    elif inEvent:
        b = [float(info[i]) for i in range(7)]
        particles.append(b)
numEv = len(events)

# update this function
def makeFourVector(momentum_magnitude, mass, theta, phi):
    """Make a four momentum vector object given the particle kinematic info
    
    Parameters
    ==========
    momentum_magnitude : float
      magnitude of the three momentum
      
    mass : float
      particle mass
      
    theta : float
      polar angle of particle momentum vector
      
    phi : float
      azimuthal angle of particle vector
      
    Returns
    =======
    fv : array
      a 4-element numpy array where component 0 is the time-like component of
      the particle's four momentum and components 1-3 are the three momentum
    """
    
    '''Your code here'''
    return fv

#update this function
def invMsq(a):
    """Compute the square of the invariant mass from a four momentum vector
    
    Parameters
    ==========
    a : array
      a four-element array representing the four momentum vector of a particle
      
    Returns
    =======
    v : float
      the squared invariant mass of the particle 
    """
    
    '''Your code here'''
    return v

# Arrays to store the invariant mass squared of 2-particle pairs
MsqKK = np.zeros(numEv)
MsqKpi = np.zeros(numEv)

# Arrays to store the mass of 2-particle pairs
MassKK = np.zeros(numEv)
MassKPi = np.zeros(numEv)

# Objects to store the four-momenta of the K+, K-,  pi+ and pi-
K1=np.zeros([numEv,4])
K2=np.zeros([numEv,4])
Pi1=np.zeros([numEv,4])

nDplus=0
nDminus=0

# This is a loop over the simulated events, each of which contains a list
# of particles.
for i in range(numEv):
    '''Your code here'''

# Plot the M(K pi) histogram
# Your code here

In [ ]:
#Write your answer here

### 1d. 

Plot the invariant mass of the the two kaons, $K^+$ and $K^-$ system. You will want to adjust the range of the $x$ axis to focus on the region around the $\phi$ mass.  Do you see the $\phi$ and is its mass roughly in the right place? (You will fit for this mass later in the problem)

In [ ]:
#Write your answer here

### 1e. 

Make the Dalitz plot for variables $m_{KK}^2$ and $m_{\pi K}^2$ (combining the kaon and the pion of the opposite charge).

In [ ]:
#Write your answer here

### 1f. 

Explain the structure you see in the Dalitz plot you have created.

Write your answer here

### 1g. 

Using these data, compute the branching fractions for decays
\begin{eqnarray*}
D_s^+ &\rightarrow &  \phi \pi^+\\
D_s^+ & \rightarrow &  \overline K^{*0}  K^+
\end{eqnarray*}
as well as the nonresonant
$$                                                                                            
D_s^+ \rightarrow K^+ K^- \pi^+                                                                
$$
and compare them to the values given in the PDG.

Note: The shape of a resonance decay is a Breit-Wigner.  The non-resonant background shape is non-trivial to predict, but if you only look in the region close to the resonant mass, approximating the background using a linear function does a reasonable job. 

In [ ]:
#Write your answer here

---

## Question 2: Angular Distributions in e+e- --> mu+mu-

### Learning objectives
In this question you will:

- Study the angular distribution of muons produced in e+e- annihilation
- Demonstrate using data collected by the Babar experiment that the two muons are produced back-to-back in both theta and phi
- Assess quantitatively the accuracy of the calculation of the angular distribution by performing a chisquare calculation


### 2a. 

Consider the process 
$$
e^+ e^- \rightarrow \mu^+ \mu^-
$$
In QED, this occurs through the s-channel production of a virtual photon.  If the energy is high enough so that the electron and muon masses are negligible, the matrix element squared can be written as:
$$
|{\cal M}| = 2 e^4 \left ( \frac{t^2+u^2}{s^2} \right )
$$
where $e$ is the electron charge and $s$, $t$ and $u$ are the Mandelstam variables.

Find the differential cross section $d\sigma/d\Omega$ in the center-of-mass frame (where $\Omega$ is $d^2/d\cos\theta d\phi$ for the outgoing $\mu^-$). Express it in terms of the fine structure constant $\alpha$, the Mandelstam variables and $\theta$, where $\theta $ is the angle between the incoming $e^-$ and the outgoing $\mu^-$.

Write your answer here

### 2b. 

The file mumu.dat contains data corresponding to an integrated luminosity of 74.674 pb$^{-1}$ collected by the BaBar experiment at the SLAC B-factory.  The file contains events from the process $e^+e^-\rightarrow \mu^+\mu^-(\gamma)$.  (where ($\gamma$) means that the event selection may allow a low-energy photon). The photon emission can be treated as a small radiative correction, which does not modify the gross properties of the $e^+e^-\rightarrow \mu^+mu^-$ process.  The data were collected at a center-of-mass energy of10.539 GeV. Note, that this is below the B B̄ production threshold.

The following code reads this data file and puts the data into a form that can be easily used in python:

In [3]:
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Parse the input file.
file = "mumu.dat"

# Inforamtion to offset needed to nterpret the info
# The order of variables is explained in the metadata at the top of the file
#
#  isBCMuMu -- a boolean bit which provides a tighter (but cleaner) selection of di-muons.  You should require this bit to be 1
#  p1Mag -- Magnitude of the momentum of the highest-momentum track (in GeV)
#  p1CosTheta -- Cosine of thepolar angle of the highest-momentum track  
#  p1Phi -- Azimuthal angle of the highest-momentum track
#  p1EmcCandEnergy -- Electromagnetic Calorimeter energy associated with the highest-momentum track. For muons, this is expected to be non-zero, but small (<1 GeV)
#  p2Mag -- Magnitude of the momentum of the second highest-momentum track (in GeV)
#  p2CosTheta -- Cosine of thepolar angle of the second highest-momentum track
#  p2Phi -- Azimuthal angle of the second highest-momentum track
#  p2EmcCandEnergy -- Electromagnetic Calorimeter energy associated with the second highest-momentum track. For muons, this is expected to be non-zero, but small (<1 GeV)
#
inMeta = False
isBCMuMu = []
p1Mag = []
p1CosTheta = []
p1Phi = []
p1EmcCandEnergy = []
p2Mag = []
p2CosTheta = []
p2Phi = []
p2EmcCandEnergy = []

inMeta = True
for line in open(file,"r"):
    line = line.strip()
    info = line.split(",")    
    if inMeta and ("<metadata>" in info[0]):
        inMeta = True
    elif inMeta and ("</metadata>" in info[0]):
        inMeta = False
    elif not inMeta:
        isBCMuMu.append(int(info[0]))
        p1Mag.append(float(info[1]))
        p1CosTheta.append(float(info[2]))
        p1Phi.append(float(info[3]))
        p1EmcCandEnergy.append(float(info[4]))
        p2Mag.append(float(info[5]))
        p2CosTheta.append(float(info[6]))
        p2Phi.append(float(info[7]))
        p2EmcCandEnergy.append(float(info[8]))


First, let's verify our statement that presence of the photon does not grossly effect the kinematics of these events.  Make a scatter plot comparing the values of $\cos(\theta)$ for the two muon candidates in each event.  Make a second scatter plot comparing the values of $\phi$.  Explain in words what these plots tell you.  Make sure you require that the isBCMuMu bit is set to 1.  In addition, to insure that both muons are in a part of the detector where they are well-measured, require $|\cos\theta|< 0.7485$ for each muon.

In [ ]:
#Write your answer here

### 2c. 

Because the electromagnetic interaction conserves parity, the angular distribution cannot contain any terms that are odd under parity inversion.  Weak interactions, however, do not conserve parity.  Since the $Z$ boson has the same quantum numbers as the photon, diagrams involving the $Z$ and the photon both contribute to the $e^+e^-\rightarrow \mu^+\mu^-$ cross section.  The interference between these two diagrams introduces a term that violates parity.  Plot the angular distribution for the $\mu^+$ and for the $\mu^-$.  Is there evidence of parity violation in these plots?

In [ ]:
#Write your answer here

### 2d. 

Physicists often characterize such parity violating effects in terms of a "forward-backward asymmetry":

$$
A_{FB} = \left( \frac{N^{\mu-}_{\cos\theta >0} - N^{\mu-}_{\cos\theta<0}}{N^{\mu-}_{\cos\theta >0} + N^{\mu-}_{\cos\theta<0}} \right )
$$

calculate $A_{FB}$ and its uncertainty for the BaBar data (note:  Because the data falls into one of two possible categories, the uncertainty follows a binomial distribution)

Fit the $\mu^- \; \cos\theta $ distribution to the form
$$
N = N_0 \left (1+A\cos\theta+B\cos^2\theta \right )
$$
(make sure you only for for the range $|\cos\theta|<0.7485$.

Do your fitted values provide a value of $A_{FB}$ that is consistent with what calculated above?

In [ ]:
#Write your answer here

### 2e. 

We can remove this parity violating term by taking the average of the angular distributions for the $\mu^+$ and the $\mu^-$:
$$
\frac{d<N>}{d\cos \theta} = 0.5 \left ( \frac{dN^{\mu+}}{d\cos\theta} + \frac{dN^{\mu-}}{\cos\theta} \right )
$$
where $N^\pm$ are the number of $\mu^+$ and $\mu^-$ respectively.  Make a histogram of this distribution and fit the angular distribution to the form 
$$
N = N_0 \left  ( 1+B \cos^2 \theta\right )
$$
Is the fitted coefficient $B$ consistent with your expectation (from theory)? 

Calculate the $\chi^2$ between your histogram and your expectation.  Turn the resulting $\chi^2$ into a fit probability.

In [ ]:
#Write your answer here